In [ ]:
#!import config/Settings.cs

In [ ]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-rc3"

In [ ]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [ ]:
// Loading models and setting up OpenAI chat completion services
var (model1, azureEndpoint1, apiKey1, gpt35TurboServiceId) = Settings.LoadFromFile(model: "gpt-35-turbo");
builder.AddAzureOpenAIChatCompletion(model1, model1, azureEndpoint1, apiKey1, serviceId: gpt35TurboServiceId);

var (model2, azureEndpoint2, apiKey2, gpt4ServiceId) = Settings.LoadFromFile(model: "gpt-4-32k");
builder.AddAzureOpenAIChatCompletion(model2, model2, azureEndpoint2, apiKey2, serviceId: gpt4ServiceId);

var kernel = builder.Build();

In [ ]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

// Set execution settings for OpenAI service
var arguments = new KernelArguments
{
    ExecutionSettings = new OpenAIPromptExecutionSettings
    {
        ServiceId = gpt4ServiceId,
        Temperature = 2,
        TopP = 1
    }
};

// Set specific arguments for the findBooks function
arguments.Add("BooksNumber", "10");
arguments.Add("YearFrom", "1900");
arguments.Add("YearTo", "2000");

# 3.1 🚀 Using Built-in SK Plugins: Loading Functions

The SK provides a set of built-in native plugins that offer powerful functionality for code analysis and manipulation. Loading functions from these plugins allows you to leverage their capabilities seamlessly within your code. Follow the steps below to harness the features of built-in native SK plugins.

In [ ]:
using Microsoft.SemanticKernel.Plugins.Core;

#pragma warning disable SKEXP0050

var timeFunctions = kernel.ImportPluginFromObject(new TimePlugin(), "time");

arguments.Add("input", "100");

var result = await kernel.InvokeAsync(timeFunctions["daysAgo"], arguments);

Console.WriteLine(result);

#pragma warning restore SKEXP0050

# 3.2 🚀 Loading and calling functions from CustomPlugin

In [ ]:
using Microsoft.SemanticKernel;

#!import Plugins/CustomPlugin/CustomPlugin.cs

In [ ]:
// Import your CustomPlugin to Semantic Kernel

#pragma warning disable SKEXP0050

var customPluginFunctions = kernel.ImportPluginFromObject(new CustomPlugin(kernel, serviceId: gpt4ServiceId));
Console.WriteLine($"{string.Join("\n", customPluginFunctions.Select(_ => $"[{_.Name}] : {_.Description}"))}");

#pragma warning restore SKEXP0050

In [ ]:
Func<string, string, Task> TranslateAsync = async (string input, string lang) => {
    // Save new message in the context variables
    arguments["input"] = input;
    arguments["lang"] = lang;
    
    var answer = await customPluginFunctions["Translate"].InvokeAsync(kernel, arguments);
};

In [ ]:
arguments["history"] = string.Empty;

In [ ]:
#pragma warning disable SKEXP0050

Func<string, Task> ChatAsync = async (string input) => {
    // Save new message in the context variables
    arguments["message"] = input;

    var answer = await customPluginFunctions["Chat"].InvokeAsync(kernel, arguments);
};

#pragma warning restore SKEXP0050

In [ ]:
var sourceInput = "Hi, I'm looking for the best historical book suggestions top-{{$BooksNumber}} from {{$YearFrom}} to {{$YearTo}}";

In [ ]:
// Translate to 'Italian' using 'Translate' function from CustomPlugin
await TranslateAsync(sourceInput, "Italian");

In [ ]:
// Translate to 'Russian' using 'Translate' function from CustomPlugin
await TranslateAsync(sourceInput, "Russian");

In [ ]:
// Translate to 'Chinese' using 'Translate' function from CustomPlugin
await TranslateAsync(sourceInput, "Chinese");

In [ ]:
await ChatAsync(sourceInput);

In [ ]:
await ChatAsync("And from these books find top-10 the most intresting facts");

In [ ]:
await ChatAsync("Describe he main idea of selected facts");

In [ ]:
Console.WriteLine($"[Chat History]: {arguments["history"]}");

# 3.3 🚀 Build-in Plugins

In [ ]:
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.0.0-rc3"

In [ ]:
using Microsoft.SemanticKernel.Plugins.Web.Google;
using Microsoft.SemanticKernel.Plugins.Web;

var (_, googleSearchEngineId, googleApiKey, _) = Settings.LoadFromFile(model: "google-connector");

#pragma warning disable SKEXP0054
using (var googleConnector = new GoogleConnector(apiKey: googleApiKey,searchEngineId: googleSearchEngineId))
{
    var google = new WebSearchEnginePlugin(googleConnector);
    kernel.ImportPluginFromObject(new WebSearchEnginePlugin(googleConnector), "google");

    var question = "What's the largest building in the world?";
    var function = kernel.Plugins["google"]["search"];
    var result = await kernel.InvokeAsync(function, new(question));
    Console.WriteLine($"Google search result: {result}");
}
